In [28]:
import numpy as np
import cvxpy as cp

In [50]:
class MyPolygon(object):
    def __init__(self):
        self.p_min = -1
        self.p_max = 1
        self.q_min = -1
        self.q_max = 1
        self.p_minus = -0.5
        self.p_plus = 0.5
        self.q_plus = 0.5
        self.q_minus = -0.5
        
        self.tau_1 = -1
        self.tau_2 = 1
        self.tau_3 = -1
        self.tau_4 = 1

        self.rho_1 = 1.5
        self.rho_2 = -1.5
        self.rho_3 = -1.5
        self.rho_4 = 1.5

        self.soc_min = 0.
        self.soc_max = 10.
        self.eff = 1.
        
        self.soc = 0.
        
def map_point_to_polygon(bounds, point, delta_t):
    
    # Inequality constraints.
    G = np.array([[-1, 0],
                  [1, 0],
                  [0, -1],
                  [0, 1],
                  [- bounds.tau_1, 1],
                  [bounds.tau_2, -1],
                  [bounds.tau_3, -1],
                  [-bounds.tau_4, 1],
                  [-1, 0],
                  [1, 0]])
    
    h = np.array([- bounds.p_min, 
                  bounds.p_max, 
                  - bounds.q_min, 
                  bounds.q_max, 
                  bounds.rho_1,
                  - bounds.rho_2,
                  - bounds.rho_3,
                  bounds.rho_4,
                  -(bounds.soc - bounds.soc_max) / (delta_t * bounds.eff),
                  bounds.eff * (bounds.soc - bounds.soc_min) / delta_t])

    # Define and solve the CVXPY problem.
    x = cp.Variable(2)
    prob = cp.Problem(cp.Minimize(cp.sum_squares(x - point)), [G @ x <= h])
    prob.solve()
    
    return x.value

In [53]:
new_point = np.array([1, 1])
bounds = MyPolygon()
x = map_point_to_polygon(bounds, new_point, 1)
print(x)

[1.39991996e-29 1.00000000e+00]


In [54]:
x[0] == 0

False